In [7]:
import numpy as np
from utils import read_canonical_model, load_pc, visualize_icp_result
import scipy

In [8]:
def sample_points(pc, num_samples=1500):
    """Downsample point cloud by selecting equally spaced points."""
    N = pc.shape[0]
    if N <= num_samples:
        return pc  # If already small, keep it as is
    indices = np.round(np.linspace(0, N - 1, num_samples)).astype(int)
    return pc[indices]

In [10]:
def icp(source, target, max_iter=50):
    '''
    Perform  ICP  to align a source point cloud to a target point cloud
    source: (N,3)
    target: (M,3)
    '''
    error = float('inf')
    pose = np.eye(4)

    for i in range(max_iter):
        tree = scipy.spatial.KDTree(target)
        dist, idx = tree.query(source)

        matched_target = target[idx]

        source_avg = np.mean(source,axis=0)
        target_avg = np.mean(matched_target,axis=0)

        s_align = source-source_avg
        t_align = matched_target-target_avg

        H = s_align.T@t_align

        U,S,Vt = np.linalg.svd(H)

        R = Vt.T @ U.T

        if np.linalg.det(R) < 0:
            Vt[-1, :] *= -1
            R = Vt.T @ U.T

        t = target_avg - R @ source_avg

        T = np.eye(4)
        T[:3,:3] = R
        T[:3,3] = t

        source = (R @ source.T).T + t
        error = np.mean(dist)

        pose = T @ pose

    return pose

In [ ]:
def disc_yaw(source_pc,target_pc,yaw_steps = 40):
    """Find the best yaw rotation by testing different angles and selecting the lowest ICP error."""
    best_angle = 0
    best_error = float('inf')
    best_transformed_pc = source_pc  

    # Discretize yaw angles from -π to π
    yaw_angles = np.linspace(-np.pi, np.pi, yaw_steps)

    target_avg = np.mean(target_pc,axis=0)
    t_align = target_pc-target_avg

    print(t_align)

    source_avg = np.mean(source_pc,axis=0)
    s_align = source_pc-source_avg

    print(s_align)

    for angle in yaw_angles:
        R = np.array([[np.cos(angle), -np.sin(angle), 0],
                  [np.sin(angle),  np.cos(angle), 0],
                  [0,             0,             1]])
        rotated_pc = (R @ s_align.T).T

        tree = scipy.spatial.KDTree(t_align)
        distances, _ = tree.query(rotated_pc)
        error = np.mean(distances**2)

        if error < best_error:
            best_error = error
            best_angle = angle
            best_transformed_pc = rotated_pc

    best_transformed_pc += target_avg 
    print(f"Best yaw angle: {np.degrees(best_angle):.2f}° with error: {best_error:.5f}")

    return best_transformed_pc, best_angle

In [ ]:
obj_name = 'drill' # drill or liq_container
num_pc = 4 # number of point clouds
num_samples = 2000

source_pc = read_canonical_model(obj_name)

for i in range(num_pc):
    # if i != 3: continue
    pose = np.eye(4)
    target_pc = load_pc(obj_name, i)
    target_pc = sample_points(target_pc, num_samples)  # Sample target cloud
    # visualize_icp_result(source_pc, target_pc, pose)

    source_pc_aligned, best_yaw = disc_yaw(source_pc, target_pc)
    # visualize_icp_result(source_pc_aligned, target_pc, pose)

    # estimated_pose, you need to estimate the pose with ICP
    pose = icp(source_pc_aligned, target_pc)

    # visualize the estimated result
    visualize_icp_result(source_pc_aligned, target_pc, pose)



[[-0.0235365   0.02550943  0.06222867]
 [-0.0235365   0.02745657  0.06125529]
 [-0.0105365   0.0164537   0.0637227 ]
 ...
 [ 0.0604635   0.01120416 -0.09194738]
 [ 0.0604635   0.00448182 -0.09194738]
 [ 0.0594635  -0.00449516 -0.0918733 ]]
[[ 0.00716002 -0.02472856 -0.10273858]
 [ 0.00716002 -0.02295774 -0.10368685]
 [ 0.00716002 -0.02844984 -0.09924242]
 ...
 [ 0.0665947   0.06129296  0.06332891]
 [ 0.06834279  0.06115418  0.06332891]
 [ 0.07183895  0.06138934  0.06158084]]
Best yaw angle: 50.77° with error: 0.00158
[[-0.009688    0.00012792  0.06318297]
 [-0.020688    0.02401589  0.06049227]
 [-0.009688   -0.00087886  0.06217639]
 ...
 [ 0.061312    0.00951163 -0.09089859]
 [ 0.060312    0.00281739 -0.0907983 ]
 [ 0.060312   -0.00395739 -0.0907983 ]]
[[ 0.00716002 -0.02472856 -0.10273858]
 [ 0.00716002 -0.02295774 -0.10368685]
 [ 0.00716002 -0.02844984 -0.09924242]
 ...
 [ 0.0665947   0.06129296  0.06332891]
 [ 0.06834279  0.06115418  0.06332891]
 [ 0.07183895  0.06138934  0.06158084